In [0]:
# 3) GOLD – aggregate into a consumption-ready fact table

from pyspark.sql.functions import date_trunc, count

# TODO 1: point at your silver & gold paths
silver_path = "/tmp/silver/doc_events/"
gold_path   = "/tmp/gold/doc_events_usage/"

# TODO 2: clear out previous gold if you need idempotence
# dbutils.fs.rm(gold_path, recurse=True)

# TODO 3: read Silver
df_silver = spark.read.format("delta").load(silver_path)

# TODO 4: choose your grain (e.g. daily per doc) and write your aggregations
gold = (
  df_silver
    .withColumn("day", date_trunc("DAY","event_ts"))
    .groupBy("day","doc_id")
    .agg(
      count("*"   ).alias("n_events"),
      count("user_id").alias("n_users")
    )
)

# TODO 5: write the Gold Δ table; consider partitionBy(…) for performance
gold.write \
  .format("delta") \
  .mode("overwrite") \
  .partitionBy("day") \
  .save(gold_path)

# sanity-check
display(spark.read.format("delta").load(gold_path))